# Tensor Contractions

In [10]:
import numpy as np
np.random.seed(47)

## Vectors, Matrices and Tensors

Generate random vectors:

In [12]:
column_vector = np.random.randn(3,1)
row_vector = np.random.randn(1,3)

In [13]:
# outer product
column_vector @ row_vector

array([[-0.54307528,  0.89442696, -1.52451812],
       [ 0.83631784, -1.37738773,  2.34770715],
       [ 0.59187369, -0.97479632,  1.66150477]])

Generate a random square matrix:

In [14]:
matrix = np.random.randn(3,3)
matrix

array([[-1.01278675,  0.82362332, -0.29465035],
       [-0.59247057, -1.39372706,  1.10417967],
       [-0.93234029, -0.94315644,  0.43660623]])

Generate a random rank 4 tensor $A_{i,j,k,l}$ where $i\in \{1,2,3\}, j\in \{1,2\}, k\in \{1,2,3,4\}, l\in \{1,2,3,4,5\}$

In [18]:
A = np.random.randn(3,2,4,5)
for label, dim in zip(["i", "j", "k","l"], A.shape):
    print(f"{label} -> dim: {dim}")

i -> dim: 3
j -> dim: 2
k -> dim: 4
l -> dim: 5


## Contraction

Define a tensor which is a combination of three tensors

$$
\Gamma_{ij} = \sum_{m,k,l}A_{imk} B^{mkl} C_{lj} 
$$

In [20]:
i, j, k, l, m = 2,3,4,5,6

A = np.random.randn(i,m,k)
B = np.random.randn(m,k,l)
C = np.random.randn(l, j)

Gamma = np.einsum("imk,mkl,lj->ij", A, B, C)

for label, dim in zip(["i", "j"], Gamma.shape):
    print(f"{label} -> dim: {dim}")

i -> dim: 2
j -> dim: 3


## Permutation and Reshape

In [22]:
A = np.random.randn(i,m,k)

Aprime = A.transpose(2,0,1)

for label, dim in zip("k,i,m".split(","), Aprime.shape):
    print(f"{label} -> dim: {dim}")

k -> dim: 4
i -> dim: 2
m -> dim: 6


In [23]:
A = np.random.randn(i,m,k)

Aprime = A.reshape(i, m*k)

for label, dim in zip("i,m*k".split(","), Aprime.shape):
    print(f"{label} -> dim: {dim}")

i -> dim: 2
m*k -> dim: 24


##  Contraction costs

Lets assume $A_{ijk}$ and $B_{ijm}$ the contraction $A_{ijk} B_{ijm}$ will cost 

$$
{\rm cost}(A\times B) = \frac{|{\rm dim}(A)||{\rm dim}(B)|}{|{\rm dim}(A\cap B)|} = \frac{{\rm dim}(i\cdot j\cdot k) {\rm dim}(i\cdot j\cdot m)}{{\rm dim}(i\cdot j)}
$$

In [25]:
A = np.random.randn(i,j,k)
B = np.random.randn(i,j,m)

AB = np.einsum("ijk,ijm->km", A, B)
cost = (i*j*k)*(i*j*m) / (i*j)
print(f"AB shape : {AB.shape}, cost: O({cost:.0f})")

AB shape : (4, 6), cost: O(144)


**How you contract matters:** $A_{ijk}B^{iln}C^j_{lm}$ lets assume dimensions $i,j,k,l,m, n$ are $D, d, d, d, D, d$

$$
cost(A\times B\times C) = d^4D^2\\
cost((A\times B)\times C) = 2d^4 D\\
cost((A\times C)\times B) = 2d^3 D^2
$$

In [28]:
from ncon import ncon
d = 10
A = np.random.rand(d,d,d); B = np.random.rand(d,d,d,d)
C = np.random.rand(d,d,d); D = np.random.rand(d,d)

TensorArray = [A,B,C,D]
IndexArray = [[1,-2,2],[-1,1,3,4],[5,3,2],[4,5]]

E = ncon(TensorArray,IndexArray,con_order = [5,3,4,1,2])
E.shape

(10, 10)